# Продвинутое машинное обучение
## Третье домашнее задание

Студент группы MADE-22

Стариков Андрей

Третье домашнее задание посвящено достаточно простой, но, надеюсь, интересной задаче, в которой потребуется творчески применить методы сэмплирования.

В этом небольшом домашнем задании мы попробуем улучшить метод Шерлока Холмса. Как известно, в рассказе The Adventure of the Dancing Men великий сыщик расшифровал загадочные письмена, которые выглядели примерно так:

![](./dancing.png)

Пользовался он для этого так называемым частотным методом: смотрел, какие буквы чаще встречаются в зашифрованных текстах, и пытался подставить буквы в соответствии с частотной таблицей: E — самая частая и так далее.

В этом задании мы будем разрабатывать более современный и продвинутый вариант такого частотного метода. В качестве корпусов текстов для подсчётов частот можете взять что угодно, но для удобства вот вам [“Война и мир” по-русски и по-английски](https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip): 

## 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:

* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2&ndash;3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [1]:
from copy import copy
import re
from zipfile import ZipFile

import numpy as np

import utils_hw3 as utils

Проверим работоспособность частотного метода на маленьком корпусе текста.  
Закодируем один абзац текста.

In [2]:
src_text = """
"Война и мир" – самый известный роман Льва Николаевича Толстого, как никакое 
другое произведение писателя, отражает глубину его мироощущения и философии.
Эта книга из разряда вечных, потому что она обо всем – о жизни и смерти, 
о любви и чести, о мужестве и героизме, о славе и подвиге, о войне и мире.
"""

src_text = utils.text_to_str(src_text, utils.ALPHABET, min_len=2)
enc_text = utils.text_encoder(src_text, char_shift=20)
print (enc_text)

ёѡэѝшічіѢчџіѠшѢђэічъёхѠјѝђэіџѡѢшѝіцќёшіѝчѥѡцшхёчюшіјѡцѠјѡўѡіѥшѥіѝчѥшѥѡхіыџщўѡхі4џѡчъёхыхѝчхі4чѠшјхцяіѡјџшѕшхјіўцщѐчѝщіхўѡіѢчџѡѡњщњхѝчяічіѓчцѡѠѡѓччіћјшіѥѝчўшічъіџшъџяышіёхюѝђљі4ѡјѡѢщіюјѡіѡѝшіѡѐѡіёѠхѢіѡіѕчъѝчічіѠѢхџјчіѡіцїѐёчічіюхѠјчіѡіѢщѕхѠјёхічіўхџѡчъѢхіѡіѠцшёхічі4ѡыёчўхіѡіёѡэѝхічіѢчџх


Получим частотные характеристики для каждой буквы текста, отсортируем в порядке убывания вероятности появления символа, создадим словарь для раскодирования.

In [3]:
src_freq = utils.calc_ngram_freq(src_text)
enc_freq = utils.calc_ngram_freq(enc_text)

src_aplabet = [x[0] for x in sorted(src_freq.items(), key=lambda x: x[1], reverse=True)]
enc_aplabet = [x[0] for x in sorted(enc_freq.items(), key=lambda x: x[1], reverse=True)]

enc_dict = dict(zip(enc_aplabet, src_aplabet))
print(enc_dict)

{'і': ' ', 'ч': 'и', 'ѡ': 'о', 'х': 'е', 'ш': 'а', 'ѝ': 'н', 'ё': 'в', 'ј': 'т', 'џ': 'р', 'Ѣ': 'м', 'Ѡ': 'с', 'ц': 'л', 'ў': 'г', 'ъ': 'з', 'ѥ': 'к', 'щ': 'у', 'э': 'й', 'ю': 'ч', 'ы': 'д', '4': 'п', 'ђ': 'ы', 'я': 'я', 'ѕ': 'ж', 'ѐ': 'б', 'њ': 'щ', 'ѓ': 'ф', 'ќ': 'ь', 'ћ': 'э', 'љ': 'х', 'ї': 'ю'}


Раскодируем и сравним с оригиналом.

In [4]:
dec_text = "".join([enc_dict[c] for c in enc_text])

print(f"Исходный текст:\n{src_text}\n")
print(f"Расшифрованный текст:\n{dec_text}")

assert src_text == dec_text

Исходный текст:
война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философии эта книга из разряда вечных потому что она обо всем о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мире

Расшифрованный текст:
война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философии эта книга из разряда вечных потому что она обо всем о жизни и смерти о любви и чести о мужестве и героизме о славе и подвиге о войне и мире


Теперь попробуем обучиться на полном тексте произведения &laquo;Война и мир&raquo;. А в качесте зашифрованного текста отрывок из А.П. Чехова.

In [5]:
archive = ZipFile('corpora.zip', 'r')
text_anna = utils.read_file_from_archive(archive, "AnnaKarenina.txt")
text_war = utils.read_file_from_archive(archive, "WarAndPeace.txt")
text_war_eng = utils.read_file_from_archive(archive, "WarAndPeaceEng.txt")

In [6]:
str_anna = utils.text_to_str(text_anna, utils.ALPHABET)
str_war = utils.text_to_str(text_war, utils.ALPHABET)
str_war_eng = utils.text_to_str(text_war_eng, utils.ALPHABET)

str_rus = str_war + str_anna

print(str_rus[:232])

война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя отражает глубину его мироощущения и философии эта книга из разряда вечных потому что она обо всем о жизни и смерти о любви и чести 


In [7]:
# А.П. Чехов
# Крыжовник
#
src_text = """
Еще с раннего утра всё небо обложили дождевые тучи; было тихо, не жарко и 
скучно, как бывает в серые пасмурные дни, когда над полем давно уже нависли 
тучи, ждешь дождя, а его нет. Ветеринарный врач Иван Иваныч и учитель гимназии 
Буркин уже утомились идти, и поле представлялось им бесконечным. Далеко 
впереди еле были видны ветряные мельницы села Мироносицкого, справа тянулся 
и потом исчезал далеко за селом ряд холмов, и оба они знали, что это берег 
реки, там луга, зеленые ивы, усадьбы, и если стать на один из холмов, то 
оттуда видно такое же громадное поле, телеграф и поезд, который издали похож 
на ползущую гусеницу, а в ясную погоду оттуда бывает виден даже город. 
Теперь, в тихую погоду, когда вся природа казалась кроткой и задумчивой, Иван 
Иваныч и Буркин были проникнуты любовью к этому полю и оба думали о том, 
как велика, как прекрасна эта страна.
"""

src_text = utils.text_to_str(src_text, utils.ALPHABET, min_len=2)
enc_text = utils.text_encoder(src_text, char_shift=20)

rus_freq = utils.calc_ngram_freq(str_rus)
enc_freq = utils.calc_ngram_freq(enc_text)

rus_aplabet = [x[0] for x in sorted(rus_freq.items(), key=lambda x: x[1], reverse=True)]
enc_aplabet = [x[0] for x in sorted(enc_freq.items(), key=lambda x: x[1], reverse=True)]

enc_dict = dict(zip(enc_aplabet, rus_aplabet))

dec_text = "".join([enc_dict[c] for c in enc_text])

print(f"Исходный текст:\n{src_text[:172]}\n")
print(f"Зашифрованный текст:\n{enc_text[:172]}\n")
print(f"Расшифрованный текст:\n{dec_text[:172]}\n")
print(f"Точность расшифровки: {utils.char_accuracy(dec_text, src_text) * 100:.2f}%")

Исходный текст:
еще с раннего утра всё небо обложили дождевые тучи было тихо не жарко и скучно как бывает в серые пасмурные дни когда над полем давно уже нависли тучи ждешь дождя а его нет

Зашифрованный текст:
хњхіѠіџшѝѝхўѡіщјџшіёѠфіѝхѐѡіѡѐцѡѕчцчіыѡѕыхёђхіјщючіѐђцѡіјчљѡіѝхіѕшџѥѡічіѠѥщюѝѡіѥшѥіѐђёшхјіёіѠхџђхі4шѠѢщџѝђхіыѝчіѥѡўышіѝшыі4ѡцхѢіышёѝѡіщѕхіѝшёчѠцчіјщючіѕыхьќіыѡѕыяішіхўѡіѝхј

Расшифрованный текст:
нцн м ваиинго ктва рмщ иньо оьсозесе лозлнрун ткые ьусо тешо ин завдо е мдкыио дад ьурант р мнвун памяквиун лие догла иал посня ларио кзн иаремсе ткые злнфч лозлж а нго инт

Точность расшифровки: 41.01%


## 2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы.
Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [8]:
ngram = 2
rus_freq = utils.calc_ngram_freq(str_rus, ngram=ngram)
enc_freq = utils.calc_ngram_freq(enc_text, ngram=ngram)

print(len(rus_freq), len(enc_freq))

854 253


In [9]:
rus_aplabet = [x[0] for x in sorted(rus_freq.items(), key=lambda x: x[1], reverse=True)]
enc_aplabet = [x[0] for x in sorted(enc_freq.items(), key=lambda x: x[1], reverse=True)]

ngram_dict = dict(zip(enc_aplabet, rus_aplabet))

dec_text = utils.apply_ngram(ngram_dict, enc_freq, enc_text)

print(f"Исходный текст:\n{src_text[:242]}\n")
print(f"Расшифрованный текст:\n{dec_text[:242]}\n")
print(f"Точность расшифровки: {utils.char_accuracy(dec_text, src_text) * 100:.2f}%")

Исходный текст:
еще с раннего утра всё небо обложили дождевые тучи было тихо не жарко и скучно как бывает в серые пасмурные дни когда над полем давно уже нависли тучи ждешь дождя а его нет ветеринарный врач иван иваныч и учитель гимназии буркин уже утомились

Расшифрованный текст:
оте к онаотр сджои в скатм семнта то лтасекуе пт о анн спсо сае иии  ао кутеп стсанан т ч в ктьуе насаегное лоо  оляи ь и нигоная  п сдде ь атьто пт о исеолалтагю и зр са ч в чтьоь рноы вонва  тва  тновао д еепол еиеь ено аегмоводде дкобжточл

Точность расшифровки: 20.14%


Нда, качество расшифровки значительно ухудшилось. Это можно объяснить тем, что количество биграмм в исходном текстовом корпусе и зашифрованном тексте очень сильно отличаются.

## 3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают.

Основная часть задания &ndash; в том, как можно их улучшить:
 - предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
 - реализуйте и протестируйте его, убедитесь, что результаты улучшились.


In [10]:
model = utils.MCMCModel(str_rus, n_gram=2, n_iter=30_000)
model.fit(enc_text)
decoded_text = model.predict(enc_text)
print("\n", decoded_text, "\n")


 еше с раннего утра всъ небо обложили дождевые тучи было тихо не жарко и скучно как бывает в серые пасмурные дни когда над полем давно уже нависли тучи ждещь дождя а его нет ветеринарный врач иван иваныч и учитель гимназии буркин уже утомились идти и поле представлялось им бесконечным далеко впереди еле были видны ветряные мельницы села мироносицкого справа тянулся и потом исчезал далеко за селом ряд холмов и оба они знали что это берег реки там луга зеленые ивы усадьбы и если стать на один из холмов то оттуда видно такое же громадное поле телеграф и поезд который издали похож на ползушую гусеницу а в ясную погоду оттуда бывает виден даже город теперь в тихую погоду когда вся природа казалась кроткой и задумчивой иван иваныч и буркин были проникнуты любовью к этому полю и оба думали о том как велика как прекрасна эта страна 



In [11]:
print(f"Точность расшифровки: {utils.char_accuracy(decoded_text, src_text) * 100:.2f}%")

Точность расшифровки: 99.52%


## 4. Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

In [12]:
model = utils.MCMCModel(str_rus, n_gram=2, n_iter=30_000)
enc_text = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'
model.fit(enc_text).predict(enc_text)

'если вы вимите норжальный или подти норжальный текст у чтого сообшения который легко продитать скорее всего вы все смелали правильно и полудите жаксижальный балл за послемнее детвертое замание курса хотя конедно я нидего не обешаю'

Смысл сообщения достаточно понятен. Более того, для нас это не четвертое, а третье задание.

## 5. Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше?
Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [13]:
model = utils.MCMCModel(str_rus, n_gram=3, n_iter=30_000)
deciphered_text_mcmc = model.fit(enc_text).predict(enc_text)
print("\n", deciphered_text_mcmc, "\n")


 если вы вимиде норчальный или пожди норчальный детсд у здого сообшения тодорый легто прожидадь сторее всего вы все смелали правильно и полужиде чатсичальный балл ка послемнее жедвердое камание турса ходя тонежно я нижего не обешаю 



In [14]:
model = utils.MCMCModel(str_rus, n_gram=4, n_iter=30_000)
deciphered_text_mcmc = model.fit(enc_text).predict(enc_text)
print("\n", deciphered_text_mcmc, "\n")


 исна ву вазати корченькую ана подта корченькую тимст г штоло соожйикая моторую нилмо продатеть смории всило ву вси сзинена преванько а понгдати чемсаченькую женн бе поснизкии дитвиртои безекаи мгрсе хотя мокидко я кадило ки ожийеы 



In [15]:
model = utils.MCMCModel(str_rus, n_gram=5, n_iter=30_000)
deciphered_text_mcmc = model.fit(enc_text).predict(enc_text)
print("\n", deciphered_text_mcmc, "\n")


 идла ку катами верболывую ала песма верболывую миндм ь ймеше деезживая немерую лишне пресамомы днерии кдише ку кди дтилола прокалыве а пельсами бондаболывую золл го педлитвии симкирмеи готоваи ньрдо чемя невисве я васише ви езижох 



Вывод: на маленьких текстах нет смысла увеличивать количество н-грамм.

In [16]:
src_text = utils.text_to_str(src_text, utils.ALPHABET, min_len=2)
enc_text = utils.text_encoder(src_text)

model = utils.MCMCModel(str_rus, n_gram=3, n_iter=30_000)
deciphered_text_mcmc = model.fit(enc_text).predict(enc_text)

print(deciphered_text_mcmc)
print(f"Точность расшифровки: {utils.char_accuracy(deciphered_text_mcmc, src_text) * 100:.2f}%")

еще с раннего утра всъ небо обложили дождевые тучи было тихо не жарко и скучно как бывает в серые пасмурные дни когда над полем давно уже нависли тучи ждешь дождя а его нет ветеринарный врач иван иваныч и учитель гимназии буркин уже утомились идти и поле представлялось им бесконечным далеко впереди еле были видны ветряные мельницы села мироносицкого справа тянулся и потом исчезал далеко за селом ряд холмов и оба они знали что это берег реки там луга зеленые ивы усадьбы и если стать на один из холмов то оттуда видно такое же громадное поле телеграф и поезд который издали похож на ползущую гусеницу а в ясную погоду оттуда бывает виден даже город теперь в тихую погоду когда вся природа казалась кроткой и задумчивой иван иваныч и буркин были проникнуты любовью к этому полю и оба думали о том как велика как прекрасна эта страна
Точность расшифровки: 99.88%


In [17]:
src_text = utils.text_to_str(src_text, utils.ALPHABET, min_len=2)
enc_text = utils.text_encoder(src_text)

model = utils.MCMCModel(str_rus, n_gram=4, n_iter=30_000)
deciphered_text_mcmc = model.fit(enc_text).predict(enc_text)

print(deciphered_text_mcmc)
print(f"Точность расшифровки: {utils.char_accuracy(deciphered_text_mcmc, src_text) * 100:.2f}%")

еще с раннего утра всъ небо обложили дождевые тучи было тихо не жарко и скучно как бывает в серые пасмурные дни когда над полем давно уже нависли тучи ждешь дождя а его нет ветеринарный врач иван иваныч и учитель гимназии буркин уже утомились идти и поле представлялось им бесконечным далеко впереди еле были видны ветряные мельницы села мироносицкого справа тянулся и потом исчезал далеко за селом ряд холмов и оба они знали что это берег реки там луга зеленые ивы усадьбы и если стать на один из холмов то оттуда видно такое же громадное поле телеграф и поезд который издали похож на ползущую гусеницу а в ясную погоду оттуда бывает виден даже город теперь в тихую погоду когда вся природа казалась кроткой и задумчивой иван иваныч и буркин были проникнуты любовью к этому полю и оба думали о том как велика как прекрасна эта страна
Точность расшифровки: 99.88%


In [18]:
src_text = utils.text_to_str(src_text, utils.ALPHABET, min_len=2)
enc_text = utils.text_encoder(src_text)

model = utils.MCMCModel(str_rus, n_gram=5, n_iter=30_000)
deciphered_text_mcmc = model.fit(enc_text).predict(enc_text)

print(deciphered_text_mcmc)
print(f"Точность расшифровки: {utils.char_accuracy(deciphered_text_mcmc, src_text) * 100:.2f}%")

еще с раннего утра всъ небо обложили дождевые тучи было тихо не жарко и скучно как бывает в серые пасмурные дни когда над полем давно уже нависли тучи ждешь дождя а его нет ветеринарный врач иван иваныч и учитель гимназии буркин уже утомились идти и поле представлялось им бесконечным далеко впереди еле были видны ветряные мельницы села мироносицкого справа тянулся и потом исчезал далеко за селом ряд холмов и оба они знали что это берег реки там луга зеленые ивы усадьбы и если стать на один из холмов то оттуда видно такое же громадное поле телеграф и поезд который издали похож на ползущую гусеницу а в ясную погоду оттуда бывает виден даже город теперь в тихую погоду когда вся природа казалась кроткой и задумчивой иван иваныч и буркин были проникнуты любовью к этому полю и оба думали о том как велика как прекрасна эта страна
Точность расшифровки: 99.88%


Переход на количество n-gramm более 3х не дало никакой прибавки в точности, текст достаточно читаем.

## 6. Бонус: какие вы можете придумать применения для этой модели?
Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

Метод МСМС может быть востребован в случаях, когда &laquo;проклятие размерности&raquo; не даст возможности для работы другим методам.  
Из практического применения можно предположить использование в качестве seq2sec переводчика.